In [2]:
import os
import sys
sys.path.append('../../kl_tools/')
import numpy as np
from astropy.units import Unit
import matplotlib.pyplot as plt
import galsim
import galsim.roman as roman

import basis
import cube
import intensity
import likelihood
import mcmc
import parameters
import priors
import utils
import velocity

# new modules
from spectrum import Spectrum
#from MockObs import Mock
%matplotlib inline

# Testing `GrismGenerator` Class

In [7]:
pars_total = {
    # priors of sampled parameters
    'priors':{
        'g1': priors.GaussPrior(0., 0.1),
        'g2': priors.GaussPrior(0., 0.1),
        'theta_int': priors.UniformPrior(-np.pi/2., np.pi/2.),
        'sini': priors.UniformPrior(0., 1.),
        'v0': priors.GaussPrior(0, 20),
        'vcirc': priors.GaussPrior(200, 10),
        'rscale': priors.UniformPrior(0, 10),
    },
    # dimension parameters for model cube
    # Note that the dimensions and scales in this parameter are irrelevant
    # to the pixel scale and number of pixels in the observed image. The 
    # model cube object is by design independent of the configurations of
    # observations. A parametric surface brightness profile is drawn on 
    # this meta-grid, then observed data will be interpolated from that. 
    'model_dimension':{
        'Nx':256,
        'Ny':256,
        'scale':0.1,# arcsec
        'lambda_range':[100, 1000],
        'lambda_res': 0.5,# nm
        'lambda_unit': 'nm'
    },
    # intensity profile, or surface brightness model for the galaxy
    'intensity':{
        'type':'inclined_exp',
        'hlr':0.5,# arcsec
    },
    # kinematics model for the galaxy
    'velocity':{
        'v_unit':Unit('km / s'),
        'r_unit':Unit('arcsec'),
    },
    # SED model for the galaxy
    'sed':{
        'template':'../../data/Simulation/GSB2.spec',
        'wave_type': 'Ang',
        'flux_type': 'flambda',
        'z': 0.0,
        'wave_range': (50, 50000), # nm
        # obs-frame continuum normalization (nm, erg/s/cm2/nm)
        'obs_cont_norm': (400, 0.),
        # a dict of line names and obs-frame flux values (erg/s/cm2)
        'lines': {'Halpha': 1e-15},
        # intrinsic linewidth in nm
        'line_sigma_int': {'Halpha': 0.5,},
    },
    # observation related parameters
    # Note that from the same galaxy and shear model, we can derive 
    # multiple types of data,
    # e.g. `photometry`, `slit_spectroscopy`, `grism` and/or `IFU`.
    # Thus 'observations' is a list of dictionaries that specify 
    # observation-wise parameters.
    # TODO: maybe think about more flexible way to describe dispersion
    'observations':[

        # Roman WFI/GRISM observation, roll angle 1
        {'inst_name':'Roman/WFI',
         'type':'grism',
         'bandpass':'../../data/Bandpass/Roman/WFI_Grism_1stOrder.dat',
         'Nx': 128,# number of pixels
         'Ny': 128,
         'pixel_scale': 0.11,# arcsec
         'R_spec':461,# at 1 micron
         # can be 'airy'/'moffat'/'kolmogorov'/'vonkarman'/'opticalpsf'
         'psf_type':'airy',
         # pass the needed params to build PSF model here
         # in case of airy, we don't need any params
         'psf_kwargs':{'fwhm':0.13},# arcsec
         'disp_ang':0.,# radian
         'diameter':240,# cm
         'exp_time':30.,# seconds
         'gain':1.,
         'noise':{'type':'ccd','sky_level':0.65*1.2,'read_noise':8.5}
         },
        # Roman WFI/GRISM observation, roll angle 2
        {'inst_name':'Roman/WFI',
         'type':'grism',
         'bandpass':'../../data/Bandpass/Roman/WFI_Grism_1stOrder.dat',
         'Nx': 128,# number of pixels
         'Ny': 128,
         'pixel_scale': 0.11,# arcsec
         'R_spec':461,# at 1 micron
         'psf_type':'airy',
         'psf_kwargs':{'fwhm':0.13},# arcsec
         'disp_ang':np.pi/2.,# radian
         'diameter':240,# cm
         'exp_time':30.,# seconds
         'gain':1.,
         'noise':{'type':'ccd','sky_level':0.65*1.2,'read_noise':8.5}
         },
        # Roman WFI/Photometry image observation, H band
        {'inst_name':'Roman/WFI',
         'type':'photometry',
         'bandpass':'../../data/Bandpass/Roman/WFI_F129.dat',
         'Nx': 128,# number of pixels
         'Ny': 128,
         'pixel_scale': 0.11,# arcsec
         'psf_type':'airy',
         'psf_kwargs':{'fwhm':0.13},# arcsec
         'diameter':240,# cm
         'exp_time':30.,# seconds
         'gain':1.,
         'noise':{'type':'ccd','sky_level':0.65*1.2,'read_noise':8.5},
         },
    ],
    'use_numba': False,
}
sp = ['g1', 'g2', 'theta_int', 'sini', 'v0', 'vcirc', 'rscale']
Nobs = len(pars_total['observations'])
print("{} observations".format(Nobs))
pars_obs = pars_total['observations'].copy()
pars = []
for i in range(Nobs):
    _p = pars_total.copy()
    _p['observations'] = pars_obs[i]
    pars.append(_p)
p = parameters.Pars(sp, pars[1])
simulator1 = cube.GrismGenerator(p)

3 observations


GalSimIncompatibleValuesError: red_limit and blue_limit are required if throughput is not a LookupTable. Values {'blue_limit': None, 'red_limit': None, 'throughput': '../../data/Bandpass/Roman/WFI_Grism_1stOrder.dat'}

In [ ]:
import random
from time import time
import galsim as gs

gs.BaseDeviate( int(time()) )